Credits:
- https://www.youtube.com/watch?v=N6Su4Hk8_-g (PyPDF2)
- https://www.youtube.com/watch?v=MjY7UUSAqqE&t=298s (pdfplumber)

In [1]:
import PyPDF2 # For numPages
import pdfplumber
from datetime import date
import requests

In [2]:
file = open("../../sampleFinancialStatements/sample2.pdf", "rb")
pdf = pdfplumber.open(file)
reader = PyPDF2.PdfFileReader(file)

### Function to calculate the change in percentage %

In [3]:
# Return the change in percentage %

def calculateChange(oldVal, newVal):
    return (newVal - oldVal) / oldVal * 100

### Function to convert a string with brackets and commas to a float

In [4]:
# Return float

def becomeNumber(string):
    thingsToReplace = [',', '(', ')', ' ']
    
    for i in thingsToReplace:
        string = string.replace(i, '')
        
    try:
        return float(string)
    
    except:
        return 0

Idea is to:
- Convert the pdf page into string for further processing
- From the string, we identify the keywords
- Get the values associated with the keywords (mindful of the years, whether the new comes first or the old)
- Calculate the change and %
- Come out with the MD&A Draft text

- Following example is assuming oldVal is the most right value, and the newVal is the one left of it, possible like page 9 of sample1.pdf, where there are 4 values instead
    - Likewise for the year detection in the heading

### Still need to do (ordered from highest relevance, aka should tackle first):
- Able to get the 'units' of the money in the financial statements (is it in millions?)
    - In the sample1.pdf case, it would be '000 for staff costs
- To be able to get out all the rows available by checking whether got valid value anot
    - Need to be able to filter out those introduction pages as well
- See whether can detect the currency anot
- Currently if the rowName have special characters like '(', the rowName result might get a little wonky
    - Can try with keyWord "asset" and the results will have some error
- See whether can detect the 'DAY' in the headings anot, so far only got months and years
- Currently, it also includes invalid results (from a different format not supported yet, but still manage to chew up some numbers idk why), so need to try to find a way to know they are invalid results and reject
- Try to incorporate OneAI API
- Try for the different format as well
- Get the reasons of the change
- If for example is transportation expenses, do we need to auto find expenses such as "bus", "grab", etc?
- Able to decide the time period based on the heading
    - To see which values come first
    - And also the criteria of "three period-to-period comparisons"
    - ~~If the headings are in different pages, then it wouldn't work~~
        - ~~Search the page before for the headings~~
- ~~Solve the date issues at printing statements~~
    - ~~Need to limit the searches for date heading to first 100 words only~~
- ~~If mulitple rows starts with the same keyWord, then it would be wrong~~
- ~~Include the increase/decrease in sum as well in the % change~~
- ~~Get usrKeyword in csv form and separate them~~

In [5]:
usrKeyword = input("(Optional) Keywords seperated by commas ").lower()
usrKeyword = usrKeyword.split(',')
for i in range(len(usrKeyword)):
    usrKeyword[i] = usrKeyword[i].strip()

(Optional) Keywords seperated by commas short-term,       financial, assets   


In [6]:
# To get all the possible names of the expenses from each row

In [7]:
found = 0     # If found = 1, means found the keyWord in PDF, if found = 0, means cannot find keyWord in PDF, need key in a new keyWord again
possibleYears = range(1900, date.today().year + 1)
# The months are  in small letters so it's not caps-sensitive
possibleMonths = ["december", "november", "october", "september", "august", "july", "june", "may", "april", "march", "february", "january"]

# Excluded '-' as rowName might possibly have '-' in its name
specialCharacters = ['`', '~', '!', '@', '#', '$', '%', '^', '&', '*', '(',')', '+', '[', ']', '{', '}', '\\', ':', ';', "'", '"', '|', ',', '<', '.', '>', '/', '?']

# Iterating all the possible keyWords in usrKeyword
for keyWord in usrKeyword:
    print("Currently checking for keyWord: '{}'".format(keyWord))
    # Iterating all pages of PDF to find keyWord
    for x in range(reader.numPages):
        pageData = pdf.pages[x]
        text = pageData.extract_text().lower()

        # Variables to keep track how to determine left values are the new one or the old one
        # KIV: Maybe can delete sameMonth & sameYear variables
    #     sameMonth = 1     # If sameMonth = 1, means need to compare by years for the time period
    #     sameYear = 1     # If sameYear = 1, means need to compare by months for the time period
        rightNew = 0     # rightNew = 1 means right value is the NEW one, rightNew = -1 means right value is the OLD one
        months = []     # months will contain the months of that particular page where keyWord is found
        years = []     # years will contain the heading of the years of that particular page where keyWord is found, [-1] is the oldYear, [-2] is the newYear

        # Checking if the keyword is found in a particular page
        for row in text.split('\n'):
            if row.startswith(keyWord):
                found = 1

                # To get the full row name that starts with keyWord
                rowName = ""
                for i in row:
                    if (i.isdigit() or i in specialCharacters):
                        break
                    else:
                        rowName += i
                rowName = rowName.strip()

                # Since the values of a financial statement is always the last few values
                # Following example is assuming oldVal is the most right value, and the newVal is the one left of it
                # Possible like page 9 of sample1.pdf, where there are 4 values instead

                # Need to get the heading as well to know which values belongs to which year / month
                # Using a for loop and work backwards into the pdf until we find a heading (rightNew != 0)

                for diffPages in range(x, -1, -1):
                    textHeading = pdf.pages[diffPages].extract_text().lower()

                    # Limit the search for headings in the first 100 words
                    for word in textHeading.split()[:100]:

                        # Trying to get the months
                        if word in possibleMonths: #and word not in months:
                            months.append(word)

                        # To determine which month is the earlier one, assuming there are only 2 months value
                        if (len(months) >= 2):

                            # Case 1, when the most right values are the OLD values
                            if (possibleMonths.index(months[-1]) > possibleMonths.index(months[-2])):
                                oldMonth = months[-1].capitalize()
                                newMonth = months[-2].capitalize()
                                sameMonth = 0
                                rightNew = -1

                            # Case 2, when the most right values are the NEW values
                            elif (possibleMonths.index(months[-1]) < possibleMonths.index(months[-2])):
                                oldMonth = months[-2].capitalize()
                                newMonth = months[-1].capitalize()
                                sameMonth = 0
                                rightNew = 1

                            # Case 3, when the months are the same, need to compare the years
                            else:
                                oldMonth = newMonth = months[-1].capitalize()
                                sameMonth = 1
                                rightNew = 0

                        # If only one month value found in heading
                        elif (len(months) == 1):
                            oldMonth = newMonth = months[-1].capitalize()
                            sameMonth = 1
                            rightNew = 0

                        # If only no month values are found in heading
                        elif (len(months) == 0):
                            oldMonth = newMonth = ""

                        # Trying to get the years
                        try:
                            if int(word) in possibleYears: #and int(word) not in years:
                                years.append(int(word))
                        except (ValueError):
                            continue

                    # Comparing the years
                    if (len(years) >= 2):

                        # Case 1: right values are the NEW values
                        if (years[-1] > years[-2]):
                            newYear = years[-1]
                            oldYear = years[-2]
                            rightNew = 1
                            sameYear = 0

                        # Case 2: right values are the OLD values
                        elif (years[-1] < years[-2]):
                            oldYear = years[-1]
                            newYear = years[-2]
                            rightNew = -1
                            sameYear = 0

                        # Case 3: both years are the same, need to compare the months
                        else:
                            oldYear = newYear = years[-1]
                            sameYear = 1

                    # If only one month value found in heading
                    elif (len(years) == 1):
                        oldYear = newYear = years[-1]
                        sameYear = 1

                    # If only no month values are found in heading
                    elif (len(years) == 0):
                        oldYear = newYear = ""         

                    # To check whether heading has been found, if found, break, else, continue finding backwards
                    if (rightNew != 0):
                        break

                # Once we know which value is the NEW one, either right or left, based on the variable rightNew
                # Checking whether that particular row has any valid values to gather
                # (by checking whether is there any digits in the string)

                valid1 = 0     # If valid = 1 by the end of the loop means at least 1 digit in the string, thus valid
                valid2 = 0
                
                try:
                    for i in row.split()[-1]:
                        if i.isdigit():
                            valid1 = 1
                    for i in row.split()[-2]:
                        if i.isdigit():
                            valid2 = 1

                    # If not valid, continue to next row to check
                    if (valid1 == 0 or valid2 == 0):
                        continue
                
                except:
                    continue

                # rightNew == 1 means right value is the NEW one
                if (rightNew == 1):
                    newVal = becomeNumber(row.split()[-1])
                    oldVal = becomeNumber(row.split()[-2])

                    # If either value returns 0 from the function, means invalid
                    if (oldVal == 0 or newVal == 0):
                        continue
                    else:
                        change = calculateChange(oldVal, newVal)

                elif (rightNew == -1):
                    oldVal = becomeNumber(row.split()[-1])
                    newVal = becomeNumber(row.split()[-2])

                    # If either value returns 0 from the function, means invalid
                    if (oldVal == 0 or newVal == 0):
                        continue
                    else:
                        change = calculateChange(oldVal, newVal)

                changeVal = newVal - oldVal

                # For error checking
    #             print("months =")
    #             print(months)
    #             print("years =")
    #             print(years)
    #             print("found = {}, sameMonth = {}, sameYear = {}, rightNew = {}, oldMonth = {}, oldYear = {}, newMonth = {}, newYear = {}".format(found, sameMonth, sameYear, rightNew, oldMonth, oldYear, newMonth, newYear))
    #             print()

                # Printing out the drafting respectively based on the different cases
                # [] means need to fill in

                try:
                    print("From page {}:".format(x + 1))

                    # If only years, but no months are detected in headings
                    if (newYear and not newMonth):
                        if (change < 0):
                            print("Our {} expenses decreased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newYear, oldVal , oldYear))

                        elif (change > 0):
                            print("Our {} expenses increased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newYear, oldVal, oldYear))

                        else:
                            print("Our {} expenses stayed the same to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, newVal, newYear, oldVal, oldYear))
                        print()

                    # If only months, but no years are detected in headings
                    if (newMonth and not newYear):
                        if (change < 0):
                            print("Our {} expenses decreased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, oldVal, oldMonth))

                        elif (change > 0):
                            print("Our {} expenses increased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, oldVal, oldMonth))

                        else:
                            print("Our {} expenses stayed the same to S$[{:.2f}] in the [{}], from S$[{:.2f}] in the [{}], as a result of [company to provide reason].".format(rowName, newVal, newMonth, oldVal, oldMonth))
                        print()

                    # If both months and years are detected in headings
                    elif (newYear and newMonth):
                        if (change < 0):
                            print("Our {} expenses decreased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{} {}], from S$[{:.2f}] in the [{} {}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, newYear, oldVal, oldMonth, oldYear))

                        elif (change > 0):
                            print("Our {} expenses increased by S${:.2f} ({:.2f})% to S$[{:.2f}] in the [{} {}], from S$[{:.2f}] in the [{} {}], as a result of [company to provide reason].".format(rowName, abs(changeVal), abs(change), newVal, newMonth, newYear, oldVal, oldMonth, oldYear))

                        else:
                            print("Our {} expenses stayed the same to S$[{:.2f}] in the [{} {}], from S$[{:.2f}] in the [{} {}], as a result of [company to provide reason].".format(rowName, newVal, newMonth, newYear, oldVal, oldMonth, oldYear))
                        print()

                except:
                    continue


    if (found == 0):
        print("Please double check the keyword to look for and try again")
        print(quit)

    else:
        print("You have reach the end of this program!")
        print()

Currently checking for keyWord: 'short-term'
From page 7:
Our short-term financial instruments expenses increased by S$13714730.00 (21.23)% to S$[78310091.00] in the [December 2020], from S$[64595361.00] in the [December 2019], as a result of [company to provide reason].

From page 7:
Our short-term financial assets at amortized cost expenses decreased by S$980218.00 (29.56)% to S$[2335630.00] in the [December 2020], from S$[3315848.00] in the [December 2019], as a result of [company to provide reason].

From page 8:
Our short-term borrowings expenses increased by S$1829767.00 (15.01)% to S$[14022898.00] in the [December 2020], from S$[12193131.00] in the [December 2019], as a result of [company to provide reason].

From page 43:
Our short-term borrowings expenses increased by S$2278386.00 (15.96)% to S$[16553429.00] in the [December 2020], from S$[14275043.00] in the [December 2019], as a result of [company to provide reason].

From page 44:
Our short-term borrowings expenses decrease

In [8]:
# Trying out OneAI API
# Possible to use this to get the currency out
for x in range(reader.numPages):
    pageData = pdf.pages[x]
    text = pageData.extract_text().lower()
    # print(text)
    # print()

    api_key = "71902c8e-8005-46a1-bcfb-bca80ef5285a"
    url = "https://api.oneai.com/api/v0/pipeline"
    headers = {
      "api-key": api_key, 
      "content-type": "application/json"
    }
    payload = {
      "input": text,
      "input_type": "article",
      "steps": [
          {
            "skill": "numbers"
          }
        ]
    }
    r = requests.post(url, json=payload, headers=headers)
    data = r.json()

    # To get the currency out
    for i in data['output'][0]['labels']:
        if (i['name'] == "MONEY"):
            print("From page {}, value = {}".format(x + 1, i['value']))

From page 7, value = millions of korean won
From page 7, value = thousands of us dollars
From page 8, value = millions of korean won
From page 8, value = thousands of us dollars
From page 9, value = thousands of us dollars
From page 10, value = thousands of us dollars
From page 11, value = millions of korean won
From page 11, value = thousands of us dollars
From page 13, value = thousands of us dollars
From page 14, value = millions of korean won
From page 15, value = thousands of us dollars
From page 16, value = millions of korean won
From page 16, value = thousands of us dollars
From page 17, value = millions of korean won
From page 17, value = thousands of us dollars
From page 39, value = 5,000
From page 40, value = korean won
From page 40, value = w1,180.46
From page 44, value = millions of korean won
From page 45, value = millions of korean won
From page 45, value = millions ofkorean won
From page 46, value = millions of korean won
From page 46, value = millions ofkorean won
From 